In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Bidirectional
from nltk.translate.bleu_score import corpus_bleu

# Generate random dataset (10 records)
data = {
    "english_sentence": [
        "hello", "how are you", "good morning", "thank you", "what is your name",
        "where are you from", "i love programming", "have a great day", "see you soon", "welcome home"
    ],
    "hindi_sentence": [
        "नमस्ते", "आप कैसे हैं", "सुप्रभात", "धन्यवाद", "आपका नाम क्या है",
        "आप कहाँ से हैं", "मुझे प्रोग्रामिंग पसंद है", "आपका दिन शुभ हो", "फिर मिलेंगे", "स्वागत है"
    ]
}
df = pd.DataFrame(data)

# Data Preprocessing
def preprocess_text(text):
    text = text.lower().strip()
    return text

df["english_sentence"] = df["english_sentence"].apply(preprocess_text)
df["hindi_sentence"] = df["hindi_sentence"].apply(preprocess_text)

# Tokenization
eng_tokenizer = Tokenizer()
hin_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(df["english_sentence"])
hin_tokenizer.fit_on_texts(df["hindi_sentence"])

eng_vocab_size = len(eng_tokenizer.word_index) + 1
hin_vocab_size = len(hin_tokenizer.word_index) + 1

eng_sequences = eng_tokenizer.texts_to_sequences(df["english_sentence"])
hin_sequences = hin_tokenizer.texts_to_sequences(df["hindi_sentence"])

max_eng_len = max(len(seq) for seq in eng_sequences)
max_hin_len = max(len(seq) for seq in hin_sequences)

eng_sequences = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
hin_sequences = pad_sequences(hin_sequences, maxlen=max_hin_len, padding='post')

X_train, X_test, y_train, y_test = train_test_split(eng_sequences, hin_sequences, test_size=0.2)

def build_model(cell_type="LSTM", bidirectional=False):
    model = Sequential()
    model.add(Embedding(input_dim=eng_vocab_size, output_dim=128, input_length=max_eng_len))
    
    if cell_type == "LSTM":
        layer = LSTM(256, return_sequences=True)
    elif cell_type == "RNN":
        layer = tf.keras.layers.SimpleRNN(256, return_sequences=True)
    elif cell_type == "GRU":
        layer = GRU(256, return_sequences=True)
    else:
        raise ValueError("Invalid cell type")
    
    if bidirectional:
        model.add(Bidirectional(layer))
    else:
        model.add(layer)
    
    model.add(Dense(hin_vocab_size, activation="softmax"))
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# Training Models
models = {
    "RNN": build_model("RNN"),
    "LSTM": build_model("LSTM"),
    "Bi-LSTM": build_model("LSTM", bidirectional=True)
}

for name, model in models.items():
    print(f"Training {name} model...")
    model.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test))  # Adjusted batch size

# Evaluate with BLEU Score
def evaluate_bleu(model, X_test, y_test):
    predictions = model.predict(X_test)
    predicted_texts = [np.argmax(pred, axis=1) for pred in predictions]
    reference_texts = [[seq] for seq in y_test]
    return corpus_bleu(reference_texts, predicted_texts)

for name, model in models.items():
    bleu = evaluate_bleu(model, X_test, y_test)
    print(f"BLEU Score for {name}: {bleu}")

# Implement teacher forcing
def train_with_teacher_forcing(model, X_train, y_train):
    y_train_shifted = np.concatenate((np.zeros((y_train.shape[0], 1)), y_train[:, :-1]), axis=1)
    model.fit(X_train, y_train_shifted, epochs=10, batch_size=2, validation_split=0.2)  # Adjusted batch size
    return model

# Train RNN with teacher forcing
rnn_tf_model = build_model("RNN")
rnn_tf_model = train_with_teacher_forcing(rnn_tf_model, X_train, y_train)

# Evaluate translation accuracy with and without teacher forcing
rnn_bleu = evaluate_bleu(models["RNN"], X_test, y_test)
rnn_tf_bleu = evaluate_bleu(rnn_tf_model, X_test, y_test)
print(f"BLEU Score without Teacher Forcing: {rnn_bleu}")
print(f"BLEU Score with Teacher Forcing: {rnn_tf_bleu}")

Training RNN model...
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.1125 - loss: 3.0706 - val_accuracy: 0.5000 - val_loss: 2.9354
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5208 - loss: 2.8672 - val_accuracy: 0.6250 - val_loss: 2.7469
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6417 - loss: 2.6759 - val_accuracy: 0.6250 - val_loss: 2.4930
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6417 - loss: 2.3072 - val_accuracy: 0.6250 - val_loss: 2.0749
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5083 - loss: 2.0628 - val_accuracy: 0.6250 - val_loss: 1.7774
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5042 - loss: 1.8434 - val_accuracy: 0.6250 - val_loss: 1.6709
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7333 - loss: 1.4248 - val_accuracy: 0.6250 - val_loss: 1.6320
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7875 - loss: 1.4196 - val_accuracy: 0.62

c:\ProgramData\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
BLEU Score for Bi-LSTM: 6.457042061640853e-78
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.0885 - loss: 3.0957 - val_accuracy: 0.1250 - val_loss: 3.1185
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7135 - loss: 2.8843 - val_accuracy: 0.1250 - val_loss: 3.1188
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8073 - loss: 2.6879 - val_accuracy: 0.2500 - val_loss: 3.1271
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8750 - loss: 2.4402 - val_accuracy: 0.2500 - val_loss: 3.1532
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8281 - loss: 2.2244 - val_accuracy: 0.2500 - val_loss: 3.2273
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8125 - loss: 1.9900 - val_accuracy: 0.2500 - val_loss: 3.3640
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7656 - loss: 1.7162 - val_accuracy: 0.2500 - val_loss: 3.5676
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 